In [1]:
from sklearn.model_selection import GridSearchCV
import json
import gc
from sklearn.kernel_ridge import KernelRidge
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import spacy
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.decomposition import TruncatedSVD
from transformers import AutoTokenizer, AutoModel ,BertForTokenClassification,TFBertForSequenceClassification
from sklearn.feature_extraction.text import CountVectorizer
#from thundersvm import SVR
from sklearn.model_selection import RandomizedSearchCV
# path_to_json = 'judgments_ECtHR/judgments_ECtHR/'
# json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
#Dumping jsons to csv
columns = ['itemid', 'docname', 'appno','conclusion','importance','originatingbody','typedescription','kpdate','extractedappno','doctypebranch','respondent','article','url','text','law']
# df = pd.DataFrame()
# for index, js in enumerate(json_files):
#     with open(os.path.join(path_to_json, js)) as json_file:
#         json_text = json.load(json_file)
#         for key,val in json_text[0].items():
#             json_text[0][key] = str(val)
#         df = df.append(json_text[0],ignore_index=True)


In [ ]:
# import pickle
# #GET NAMED ENTITIES AND DUMP THEM
# import spacy
# spacy_nlp.disable_pipes('tagger')
# spacy_nlp = spacy.load('C:\\Users\\sichi\\PycharmProjects\\NLPproiect\\venv\\Lib\\site-packages\\en_core_web_sm\\en_core_web_sm-2.3.1')
# tokens = []
# spacy_nlp.max_length = 679359709
# texts = df['text'].apply(lambda x: ' '.join([word.text for word in spacy_nlp.tokenizer(x)]))
# for text in texts:
#     tokens.append(set([ent.text for ent in spacy_nlp(text) if ent.ent_type_ == 'PERSON']))
# print(tokens)
# s = set()
# for token in tokens:
#     s = s.union(token)
# print(s)

# with open("ners.txt", "wb") as fp:
#     pickle.dump(set(s), fp)


In [2]:

spacy_nlp = spacy.load('C:\\Users\\sichi\\PycharmProjects\\NLPproiect\\venv\\Lib\\site-packages\\en_core_web_sm\\en_core_web_sm-2.3.1')
months = ["january","february","march","april","may","june","july","august","september","october","november","december"]

df = pd.DataFrame()
df = pd.read_csv('input.csv',usecols=["kpdate", "appno", "extractedappno","text","law"], dtype={col:str for col in df.columns})
years = [str(i) for i in range(1960,2021)]
with open("ners2.txt", "rb") as fp:
    ners = set(pickle.load(fp))

def clean_numbers(row,col):
   # year = row['kpdate'].split('-')[0]
    article = row['appno']
    if article is None:
        article = row['extractedappno']
    for year in years:
        row[col] = row[col].replace(year,'YEAR')
    for x in article.split(';'):
        row[col] = row[col].replace(x,'APPNUMBER')
    if(re.search("^\s*[0-9]+\.",str(row['law']))):
        app_n = str(row['law']).split('.')[0] + '.'
        row[col] = row[col].replace(app_n,'NUMBER')
    return row[col]
        
df['text'] = df['text'].apply(lambda x: x.lower())
for index, row in df.iterrows():
    df.at[index, 'text'] = clean_numbers(row,'text')
df['text'] = df['text'].apply(lambda x: spacy_nlp.tokenizer(x))
df['text'] = df['text'].apply(lambda sentence: ' '.join([token.lemma_ for token in sentence if token.is_stop == False and token.is_punct == False and token.text not in ners and token.text not in years]))
#and len(token.text) > 2

# df['law'] = df['law'].apply(lambda x: str(x).lower())
# for index, row in df.iterrows():
#     df.at[index, 'law'] = clean_numbers(row,'law')
# df['law'] = df['law'].apply(lambda x: spacy_nlp.tokenizer(x))
# df['law'] = df['law'].apply(lambda sentence: ' '.join([token.lemma_ for token in sentence if token.is_stop == False and token.is_punct == False]))

df['kpdate'] = df['kpdate'].apply(lambda date: float(int(date.split('-')[0]) + float((int(date.split('-')[1]) - 1)/12)))

In [ ]:
def calculate_special_mae(predicted):
    s = 0
    for i,y in enumerate(predicted):
        if( (y_test.values[i] < 2000 and predicted[i] > 2000 ) or (y_test.values[i] > 2000 and predicted[i] < 2000 )):
            print("DAMN:" + str(y_test.values[i]) + ' ' + str(predicted[i]))
        if(y_test.values[i] < 2000 and predicted[i] < 2000):
            diff = 0
        else:
            diff = abs(y_test.values[i] - predicted[i])
            s = s + diff

    print(str(s/len(y_test)))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,2),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [9]:
svr = LinearSVR(C=100) #13 for 1-gram
svr.fit(X_train,y_train)
predicted = svr.predict(X_test)

mean = mean_absolute_error(y_test, predicted)
print(str(mean)) #1.8510149964071645

102 1.8543067647875409
102 1.8901811037647014
102 1.928351069981783
102 1.968560433333985
102 2.0095345255195913
102 2.0522464681770747
102 2.0959153018667807
102 2.1403776690778082
102 2.1857679458458166
102 2.2319767392774783


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,1),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [ ]:
svr = LinearSVR(C=99) #13 for 1-gram
svr.fit(X_train,y_train)
predicted = svr.predict(X_test)

mean = mean_absolute_error(y_test, predicted)
print(str(mean)) #1.5862547589534721

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,1),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [ ]:
import numpy as np
clf = KernelRidge(kernel='rbf',alpha=0.005,gamma = 0.05)
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
mean = mean_absolute_error(y_test, predicted)
print(gamm,mean) #1.311677646656628

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,2),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [ ]:
import numpy as np
clf = KernelRidge(kernel='rbf',alpha=0.005,gamma = 0.02)
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
mean = mean_absolute_error(y_test, predicted)
print(gamm,mean) #1.109751932425466

In [ ]:
pd.Series(abs(svr.coef_[0]), tf_vectorizer.get_feature_names()).nlargest(50).plot(kind='barh')

In [ ]:
def plot_coefficients(classifier, feature_names, top_features=100):
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    # create plot
    plt.figure(figsize=(100, 10))
    feature_names = np.array(feature_names)
    plt.bar(np.arange(top_features), coef[top_negative_coefficients], color=['red'] * (top_features + 1))
    plt.xticks(np.arange(1, 1 + top_features), feature_names[top_negative_coefficients], rotation=60, ha='right')
    plt.show()
    
    plt.figure(figsize=(100, 10))
    plt.bar(np.arange(top_features), coef[top_positive_coefficients], color=['blue'] * ( top_features + 1))
    plt.xticks(np.arange(1, 1 + top_features), feature_names[top_positive_coefficients], rotation=60, ha='right')
    plt.show()
plot_coefficients(svr, tf_vectorizer.get_feature_names())

In [ ]:
feature_names = tf_vectorizer.get_feature_names() 
coefs_with_fns = sorted(zip(svr.coef_, feature_names)) 
df=pd.DataFrame(coefs_with_fns)
df.columns='coefficient','word'
df = df.sort_values(by='coefficient',ascending=False)


In [ ]:
df.to_csv("best_features_text.csv")

In [ ]:
gc.collect()
tf_vectorizer=TfidfVectorizer(ngram_range=(1,2),encoding='utf-8',sublinear_tf=True,min_df=2,max_df=0.9) #min_df=0 , #binary maybe sublinear_tf=true
svd = TruncatedSVD(n_components=600)
X=svd.fit_transform(tf_vectorizer.fit_transform(text))
X_train, X_test, y_train, y_test = train_test_split(X, dates, test_size=0.1, random_state=0)


In [ ]:
from sklearn.svm import SVR
clf = SVR(kernel='rbf',C=40,gamma = 3)
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
mean = mean_absolute_error(y_test, predicted)
print(str(mean)) #0.8359267099173915

In [ ]:
from sklearn.svm import SVR
#from thundersvm import SVR
import numpy as np
params = {'kernel': ['rbf'],
          'C': np.logspace(1, 50, 17),
          'gamma': np.logspace(1, 15, 17)}
# for c in np.linspace(10,40,30):
#     for gam in np.linspace(2,12,10):
#clf = SVR(kernel='rbf',C=c,gamma = gam)

grid = RandomizedSearchCV(estimator=SVR(), param_distributions=params,verbose=2,scoring='neg_mean_absolute_error')
grid.fit(X, df['kpdate'])
print("Best parameter values:", grid.best_params_)
print("CV Score with best parameter values:", grid.best_score_)
#clf.fit(X_train,y_train)
#Best parameter values: {'kernel': 'rbf', 'gamma': 10.0, 'C': 2.7384196342643614e+22}

# predicted = clf.predict(X_test)
# mean = mean_absolute_error(y_test, predicted)
# print(str(c),str(gam),str(mean))
#calculate_special_mae(predicted) #0.7128941956283799

In [ ]:
param_grid = {
                  'kernel':['rbf'],
                  'C':np.linspace(5,30,10),
                  'gamma':np.linspace(1,10,5),
                  'epsilon':np.linspace(0.1,1,5)
                }

grid = GridSearchCV(estimator=SVR(),param_grid=param_grid,verbose=2,scoring='neg_mean_absolute_error')
grid.fit(X, df['kpdate'])
print("Best parameter values:", grid.best_params_)
print("CV Score with best parameter values:", grid.best_score_)

In [ ]:
print("Best parameter values:", grid.best_params_)
print("CV Score with best parameter values:", grid.best_score_)
# Best parameter values: {'C': 30.0, 'epsilon': 0.1, 'gamma': 10.0, 'kernel': 'rbf'}
# CV Score with best parameter values: -0.8477903605840826

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
                  'kernel':['rbf'],
                  'C':np.linspace(100,1000,5),
                  'gamma':np.linspace(25,35,5),
                }

grid = GridSearchCV(estimator=SVR(),param_grid=param_grid,verbose=2,scoring='neg_mean_absolute_error')
grid.fit(X, df['kpdate'])
print("Best parameter values:", grid.best_params_)
print("CV Score with best parameter values:", grid.best_score_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['law'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,1),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [ ]:
clf = KernelRidge(kernel='rbf',alpha=0.041,gamma = 0.015) #0.015
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
mean = mean_absolute_error(y_test, predicted)
print(mean) #1.5281275405560835
calculate_special_mae(predicted) #1.3996126603349304

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("berts/legal-bert-base-uncased") #dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
model = AutoModel.from_pretrained("berts/legal-bert-base-uncased")
model.eval()

In [ ]:
import spacy
import re
import pickle
import pandas as pd
import torch
#spacy_nlp = spacy.load('C:\\Users\\sichi\\PycharmProjects\\NLPproiect\\venv\\Lib\\site-packages\\en_core_web_sm\\en_core_web_sm-2.3.1')

df = pd.DataFrame()
df = pd.read_csv('input.csv', dtype={col:str for col in df.columns})

def clean_numbers(row,col):
    year = row['kpdate'].split('-')[0]
    article = row['appno']
    if article is None:
        article = row['extractedappno']
    row[col] = row[col].replace(year,'YEAR')
    for x in article.split(';'):
        row[col] = row[col].replace(x,'APPNUMBER')
    if(re.search("^\s*[0-9]+\.",str(row['law']))):
        app_n = str(row['law']).split('.')[0] + '.'
        row[col] = row[col].replace(app_n,'NUMBER')
    return row[col]

def sliding_window(input_ids):
    #this bert only support a max length of 512 words
    to_return = []
    for slice_list in torch.split(input_ids,512,dim = 1):
        tens = model(slice_list)[0][:,0,:].detach().numpy()
        to_return.append(tens)
    print(len(to_return))
    return numpy.concatenate(to_return) #, dim=1

# df['text'] = df['text'].apply(lambda x: x.lower())
# for index, row in df.iterrows():
#     df.at[index, 'text'] = clean_numbers(row,'text')
#df['text'] = df['text'].apply(lambda x: spacy_nlp.tokenizer(x))
#df['text'] = df['text'].apply(lambda sentence: ' '.join([token.text for token in sentence if token.is_stop == False and token.is_punct == False and token.text not in ners]))

#df['text'] = df['text'].apply(lambda x: tokenizer.encode_plus(x,return_tensors = 'pt', padding=True,pad_to_multiple_of=128))
#df['text'] = df['text'].apply(lambda x: sliding_window(x['input_ids'])) # **x


df['law'] = df['law'].apply(lambda x: str(x).lower())
for index, row in df.iterrows():
    df.at[index, 'law'] = clean_numbers(row,'law')
df['law'] = df['law'].apply(lambda x: tokenizer.encode_plus(x,return_tensors = 'pt', padding=True,pad_to_multiple_of=128))
df['law'] = df['law'].apply(lambda x: sliding_window(x['input_ids'])) # **x

In [ ]:
import spacy
import re
import pickle
import pandas as pd
import torch
#spacy_nlp = spacy.load('C:\\Users\\sichi\\PycharmProjects\\NLPproiect\\venv\\Lib\\site-packages\\en_core_web_sm\\en_core_web_sm-2.3.1')

df = pd.DataFrame()
df = pd.read_csv('input.csv', dtype={col:str for col in df.columns})

def clean_numbers(row,col):
    year = row['kpdate'].split('-')[0]
    article = row['appno']
    if article is None:
        article = row['extractedappno']
    row[col] = row[col].replace(year,'YEAR')
    for x in article.split(';'):
        row[col] = row[col].replace(x,'APPNUMBER')
    if(re.search("^\s*[0-9]+\.",str(row['law']))):
        app_n = str(row['law']).split('.')[0] + '.'
        row[col] = row[col].replace(app_n,'NUMBER')
    return row[col]

def sliding_window(input_ids):
    #this bert only support a max length of 512 words
    to_return = []
    for slice_list in torch.split(input_ids,512,dim = 1):
        tens = model(slice_list)[0][:,0,:].detach().numpy()
        to_return.append(tens)
    print(len(to_return))
    return numpy.concatenate(to_return) #, dim=1

# df['text'] = df['text'].apply(lambda x: x.lower())
# for index, row in df.iterrows():
#     df.at[index, 'text'] = clean_numbers(row,'text')
#df['text'] = df['text'].apply(lambda x: spacy_nlp.tokenizer(x))
#df['text'] = df['text'].apply(lambda sentence: ' '.join([token.text for token in sentence if token.is_stop == False and token.is_punct == False and token.text not in ners]))

#df['text'] = df['text'].apply(lambda x: tokenizer.encode_plus(x,return_tensors = 'pt', padding=True,pad_to_multiple_of=128))
#df['text'] = df['text'].apply(lambda x: sliding_window(x['input_ids'])) # **x


df['law'] = df['law'].apply(lambda x: str(x).lower())
for index, row in df.iterrows():
    df.at[index, 'law'] = clean_numbers(row,'law')
df['law'] = df['law'].apply(lambda x: tokenizer.encode_plus(x,return_tensors = 'pt', padding=True,pad_to_multiple_of=128))
df['law'] = df['law'].apply(lambda x: sliding_window(x['input_ids'])) # **x

In [ ]:
import torch
def sliding_window(input_ids):
    #this bert only support a max length of 512 words
    to_return = []
    for slice_list in torch.split(input_ids,512,dim = 1):
        tens = model(slice_list)[0][:,0,:].detach().numpy() #last_hidden_state[0]
        to_return.append(tens)
    print(len(to_return))
    #return torch.cat(to_return) #, dim=1
    return numpy.concatenate(tens)

df['text'] = df['text'].apply(lambda x: sliding_window(x['input_ids'])) # **x


In [ ]:
def sliding_window(input_ids):
    #this bert only support a max length of 512 words
    to_return = []
    for i in range(0,int(len(input_ids)/512),1):
        slice_list = input_ids[512 * i,512*(i+1)]
        to_return.append(model(slice_list))
    return to_return

df['law'] = df['law'].apply(lambda x: sliding_window(x['input_ids']))